In [1]:
# Uncomment and run the commands below if imports fail
# !conda install numpy pytorch torchvision cpuonly -c pytorch -y
# !pip install matplotlib --upgrade --quiet
!pip install jovian --upgrade --quiet

     |████████████████████████████████| 92kB 2.4MB/s 


In [0]:
import torch
import jovian
import torchvision
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, TensorDataset, random_split

In [0]:
project_name='02-insurance-linear-regression' # will be used by jovian.commit

## Step 1: Download and explore the data

Let us begin by downloading the data. We'll use the `download_url` function from PyTorch to get the data as a CSV (comma-separated values) file. 

In [83]:
DATASET_URL = "https://hub.jovian.ml/wp-content/uploads/2020/05/insurance.csv"
DATA_FILENAME = "insurance.csv"
download_url(DATASET_URL, '.')

Using downloaded and verified file: ./insurance.csv


To load the dataset into memory, we'll use the `read_csv` function from the `pandas` library. The data will be loaded as a Pandas dataframe. See this short tutorial to learn more: https://data36.com/pandas-tutorial-1-basics-reading-data-files-dataframes-data-selection/

In [84]:
dataframe_raw = pd.read_csv(DATA_FILENAME)
dataframe_raw.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


We're going to do a slight customization of the data, so that you every participant receives a slightly different version of the dataset. Fill in your name below as a string (enter at least 5 characters)

In [0]:
your_name = 'PhilOrfa'

The `customize_dataset` function will customize the dataset slightly using your name as a source of random numbers.

In [0]:
def customize_dataset(dataframe_raw, rand_str):
    dataframe = dataframe_raw.copy(deep=True)
    # drop some rows
    dataframe = dataframe.sample(int(0.95*len(dataframe)), random_state=int(ord(rand_str[0])))
    # scale input
    dataframe.bmi = dataframe.bmi * ord(rand_str[1])/100.
    # scale target
    dataframe.charges = dataframe.charges * ord(rand_str[2])/100.
    # drop column
    if ord(rand_str[3]) % 2 == 1:
        dataframe = dataframe.drop(['region'], axis=1)
    return dataframe

In [87]:
dataframe = customize_dataset(dataframe_raw, your_name)
dataframe.head()

,age,sex,bmi,children,smoker,region,charges
240,23,female,38.1368,2,yes,northeast,40437.209715
651,53,female,41.1840,1,no,southeast,11108.696550
761,23,male,36.6080,1,no,southwest,2537.802750
836,36,male,32.7600,0,no,southwest,4622.344650
1147,20,female,33.1968,0,no,northwest,2374.647240


Let us answer some basic questions about the dataset. 


**Q: How many rows does the dataset have?**

In [88]:
num_rows = len(dataframe.index)
print(num_rows)

1271


**Q: How many columns doe the dataset have**

In [89]:
num_cols = len(dataframe.columns)
print(num_cols)

7


**Q: What are the column titles of the input variables?**

In [0]:
input_cols = dataframe.columns.values[0:6].tolist()

**Q: Which of the input columns are non-numeric or categorial variables ?**

Hint: `sex` is one of them. List the columns that are not numbers.

In [0]:
categorical_cols = dataframe.select_dtypes(exclude=['int', 'float']).columns.tolist()

**Q: What are the column titles of output/target variable(s)?**

In [0]:
output_cols = dataframe.columns.values[6:7].tolist()

**Q: (Optional) What is the minimum, maximum and average value of the `charges` column? Can you show the distribution of values in a graph?**
Use this data visualization cheatsheet for referece: https://jovian.ml/aakashns/dataviz-cheatsheet

In [95]:
print('Minimum=',dataframe.min(axis=0)['charges'])
print('Maximum=',dataframe.max(axis=0)['charges'])
print('Average=',dataframe.mean(axis=0)['charges'])

Minimum= 1177.967595
Maximum= 66958.9494105
Average= 14042.096754098315


Remember to commit your notebook to Jovian after every step, so that you don't lose your work.

In [0]:
jovian.commit(project=project_name, environment=None)

## Step 2: Prepare the dataset for training

We need to convert the data from the Pandas dataframe into a PyTorch tensors for training. To do this, the first step is to convert it numpy arrays. If you've filled out `input_cols`, `categorial_cols` and `output_cols` correctly, this following function will perform the conversion to numpy arrays.

In [0]:
def dataframe_to_arrays(dataframe):
    # Make a copy of the original dataframe
    dataframe1 = dataframe.copy(deep=True)
    # Convert non-numeric categorical columns to numbers
    for col in categorical_cols:
        dataframe1[col] = dataframe1[col].astype('category').cat.codes
    # Extract input & outupts as numpy arrays
    inputs_array = dataframe1[input_cols].to_numpy()
    targets_array = dataframe1[output_cols].to_numpy()
    return inputs_array, targets_array

Read through the [Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/categorical.html) to understand how we're converting categorical variables into numbers.

In [97]:
inputs_array, targets_array = dataframe_to_arrays(dataframe)
inputs_array, targets_array

(array([[23.    ,  0.    , 38.1368,  2.    ,  1.    ,  0.    ],
        [53.    ,  0.    , 41.184 ,  1.    ,  0.    ,  2.    ],
        [23.    ,  1.    , 36.608 ,  1.    ,  0.    ,  3.    ],
        ...,
        [59.    ,  0.    , 38.2356,  1.    ,  1.    ,  0.    ],
        [53.    ,  0.    , 34.58  ,  0.    ,  0.    ,  0.    ],
        [48.    ,  0.    , 34.6632,  0.    ,  0.    ,  2.    ]]),
 array([[40437.209715 ],
        [11108.69655  ],
        [ 2537.80275  ],
        ...,
        [50291.6309175],
        [11093.128725 ],
        [ 8697.864735 ]]))

**Q: Convert the numpy arrays `inputs_array` and `targets_array` into PyTorch tensors. Make sure that the data type is `torch.float32`.**

In [0]:
inputs = torch.from_numpy(inputs_array).float()
targets = torch.from_numpy(targets_array).float()

In [99]:
inputs.dtype, targets.dtype

(torch.float32, torch.float32)

Next, we need to create PyTorch datasets & data loaders for training & validation. We'll start by creating a `TensorDataset`.

In [0]:
dataset = TensorDataset(inputs, targets)

**Q: Pick a number between `0.1` and `0.2` to determine the fraction of data that will be used for creating the validation set. Then use `random_split` to create training & validation datasets. **

In [0]:
val_percent = 0.17 # between 0.1 and 0.2
val_size = int(num_rows * val_percent)
train_size = num_rows - val_size


train_ds, val_ds = random_split(dataset, [train_size, val_size])

Finally, we can create data loaders for training & validation.

**Q: Pick a batch size for the data loader.**

In [0]:
batch_size = 16

In [0]:
train_loader = DataLoader(train_ds, batch_size, shuffle=True)
val_loader = DataLoader(val_ds, batch_size)

Let's look at a batch of data to verify everything is working fine so far.

In [104]:
for xb, yb in train_loader:
    print("inputs:", xb)
    print("targets:", yb)
    break

inputs: tensor([[36.0000,  0.0000, 26.9360,  1.0000,  0.0000,  3.0000],
        [18.0000,  0.0000, 40.7264,  0.0000,  0.0000,  2.0000],
        [45.0000,  0.0000, 34.4240,  0.0000,  0.0000,  3.0000],
        [56.0000,  1.0000, 22.9840,  0.0000,  0.0000,  3.0000],
        [48.0000,  1.0000, 30.7840,  0.0000,  0.0000,  3.0000],
        [35.0000,  0.0000, 35.4692,  3.0000,  1.0000,  1.0000],
        [56.0000,  1.0000, 34.9752,  0.0000,  1.0000,  1.0000],
        [28.0000,  0.0000, 36.1608,  0.0000,  0.0000,  1.0000],
        [40.0000,  0.0000, 30.4720,  4.0000,  0.0000,  3.0000],
        [27.0000,  0.0000, 31.8136,  1.0000,  0.0000,  0.0000],
        [45.0000,  0.0000, 29.7440,  2.0000,  0.0000,  2.0000],
        [50.0000,  0.0000, 28.1580,  1.0000,  0.0000,  0.0000],
        [22.0000,  1.0000, 27.9136,  0.0000,  0.0000,  2.0000],
        [59.0000,  1.0000, 31.0232,  3.0000,  1.0000,  0.0000],
        [59.0000,  1.0000, 25.6880,  0.0000,  0.0000,  0.0000],
        [18.0000,  0.0000, 27.36

Let's save our work by committing to Jovian.

In [0]:
jovian.commit(project=project_name, environment=None)

## Step 3: Create a Linear Regression Model

Our model itself is a fairly straightforward linear regression (we'll build more complex models in the next assignment). 


In [0]:
input_size = len(input_cols)
output_size = len(output_cols)

**Q: Complete the class definition below by filling out the constructor (`__init__`), `forward`, `training_step` and `validation_step` methods.**

Hint: Think carefully about picking a good loss fuction (it's not cross entropy). Maybe try 2-3 of them and see which one works best. See https://pytorch.org/docs/stable/nn.functional.html#loss-functions

In [0]:
class InsuranceModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(input_size,output_size)                  # fill this (hint: use input_size & output_size defined above)
        
    def forward(self, xb):
        out = self.linear(xb)                         # fill this
        return out
    
    def training_step(self, batch):
        inputs, targets = batch 
        # Generate predictions
        out = self(inputs)          
        # Calcuate loss
        loss = F.l1_loss(out, targets)                         # fill this
        return loss
    
    def validation_step(self, batch):
        inputs, targets = batch
        # Generate predictions
        out = self(inputs)
        # Calculate loss
        loss = F.l1_loss(out, targets)                           # fill this    
        return {'val_loss': loss.detach()}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        return {'val_loss': epoch_loss.item()}
    
    def epoch_end(self, epoch, result, num_epochs):
        # Print result every 20th epoch
        if (epoch+1) % 20 == 0 or epoch == num_epochs-1:
            print("Epoch [{}], val_loss: {:.4f}".format(epoch+1, result['val_loss']))

Let us create a model using the `InsuranceModel` class. You may need to come back later and re-run the next cell to reinitialize the model, in case the loss becomes `nan` or `infinity`.

In [0]:
model = InsuranceModel()

Let's check out the weights and biases of the model using `model.parameters`.

In [108]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.3491, -0.0598, -0.1900, -0.1705,  0.0172,  0.2953]],
        requires_grad=True), Parameter containing:
 tensor([0.1338], requires_grad=True)]

One final commit before we train the model.

In [109]:
jovian.commit(project=project_name, environment=None)

[jovian] Error: Failed to detect Jupyter notebook or Python script. Skipping..


## Step 4: Train the model to fit the data

To train our model, we'll use the same `fit` function explained in the lecture. That's the benefit of defining a generic training loop - you can use it for any problem.

In [0]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    history = []
    optimizer = opt_func(model.parameters(), lr)
    for epoch in range(epochs):
        # Training Phase 
        for batch in train_loader:
            loss = model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        # Validation phase
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result, epochs)
        history.append(result)
    return history

**Q: Use the `evaluate` function to calculate the loss on the validation set before training.**

In [159]:
result = evaluate(model,val_loader)
print(result)

{'val_loss': 13731.7822265625}



We are now ready to train the model. You may need to run the training loop many times, for different number of epochs and with different learning rates, to get a good result. Also, if your loss becomes too large (or `nan`), you may have to re-initialize the model by running the cell `model = InsuranceModel()`. Experiment with this for a while, and try to get to as low a loss as possible.

**Q: Train the model 4-5 times with different learning rates & for different number of epochs.**

Hint: Vary learning rates by orders of 10 (e.g. `1e-2`, `1e-3`, `1e-4`, `1e-5`, `1e-6`) to figure out what works.

In [160]:
epochs = 200
lr = 1e-2
history1 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 7863.7485
Epoch [40], val_loss: 8791.5625
Epoch [60], val_loss: 7786.3638
Epoch [80], val_loss: 7874.1660
Epoch [100], val_loss: 7233.7949
Epoch [120], val_loss: 7042.3555
Epoch [140], val_loss: 7096.3037
Epoch [160], val_loss: 6597.0044
Epoch [180], val_loss: 7318.9395
Epoch [200], val_loss: 6648.0986


In [161]:
epochs = 10000
lr = 1e-2
history2 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6085.2778
Epoch [40], val_loss: 7110.5752
Epoch [60], val_loss: 8165.6602
Epoch [80], val_loss: 6186.2461
Epoch [100], val_loss: 8708.2764
Epoch [120], val_loss: 8405.2314
Epoch [140], val_loss: 7624.2085
Epoch [160], val_loss: 9315.3682
Epoch [180], val_loss: 8317.5596
Epoch [200], val_loss: 7616.9058
Epoch [220], val_loss: 5906.6040
Epoch [240], val_loss: 5764.6841
Epoch [260], val_loss: 5677.6455
Epoch [280], val_loss: 8206.3555
Epoch [300], val_loss: 5694.6040
Epoch [320], val_loss: 7406.4639
Epoch [340], val_loss: 7054.6670
Epoch [360], val_loss: 5488.0493
Epoch [380], val_loss: 8373.7344
Epoch [400], val_loss: 5526.0811
Epoch [420], val_loss: 8652.9482
Epoch [440], val_loss: 6988.8872
Epoch [460], val_loss: 5887.0078
Epoch [480], val_loss: 6450.8569
Epoch [500], val_loss: 6648.9697
Epoch [520], val_loss: 7412.9692
Epoch [540], val_loss: 5200.8442
Epoch [560], val_loss: 8629.8398
Epoch [580], val_loss: 5742.8892
Epoch [600], val_loss: 8489.0869
Epoch [620], v

KeyboardInterrupt: ignored

In [142]:
epochs = 70
lr = 1e-3
history3 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6768.6743
Epoch [40], val_loss: 6764.9082
Epoch [60], val_loss: 6761.1919
Epoch [70], val_loss: 6759.3608


In [143]:
epochs = 150
lr = 1e-4
history4 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6759.0034
Epoch [40], val_loss: 6758.6523
Epoch [60], val_loss: 6758.3003
Epoch [80], val_loss: 6757.9385
Epoch [100], val_loss: 6757.5659
Epoch [120], val_loss: 6757.1963
Epoch [140], val_loss: 6756.8301
Epoch [150], val_loss: 6756.6489


In [144]:
epochs = 200
lr = 1e-5
history5 = fit(epochs, lr, model, train_loader, val_loader)

Epoch [20], val_loss: 6756.6123
Epoch [40], val_loss: 6756.5742
Epoch [60], val_loss: 6756.5376
Epoch [80], val_loss: 6756.5000
Epoch [100], val_loss: 6756.4634
Epoch [120], val_loss: 6756.4258
Epoch [140], val_loss: 6756.3877
Epoch [160], val_loss: 6756.3506
Epoch [180], val_loss: 6756.3140
Epoch [200], val_loss: 6756.2759


**Q: What is the final validation loss of your model?**

In [162]:
val_loss = evaluate(model,val_loader)
print(val_loss)

{'val_loss': 7009.94189453125}


Let's log the final validation loss to Jovian and commit the notebook

In [0]:
jovian.log_metrics(val_loss=val_loss)

In [0]:
jovian.commit(project=project_name, environment=None)

Now scroll back up, re-initialize the model, and try different set of values for batch size, number of epochs, learning rate etc. Commit each experiment and use the "Compare" and "View Diff" options on Jovian to compare the different results.

## Step 5: Make predictions using the trained model

**Q: Complete the following function definition to make predictions on a single input**

In [0]:
def predict_single(input, target, model):
    inputs = input.unsqueeze(0)
    predictions = model(inputs)                # fill this
    prediction = predictions[0].detach()
    print("Input:", input)
    print("Target:", target)
    print("Prediction:", prediction)

In [164]:
input, target = val_ds[0]
predict_single(input, target, model)

Input: tensor([36.0000,  1.0000, 29.1460,  1.0000,  1.0000,  0.0000])
Target: tensor([21812.3086])
Prediction: tensor([34924.7422])


In [165]:
input, target = val_ds[10]
predict_single(input, target, model)

Input: tensor([40.0000,  1.0000, 27.3676,  1.0000,  0.0000,  1.0000])
Target: tensor([6708.8467])
Prediction: tensor([13325.1250])


In [166]:
input, target = val_ds[23]
predict_single(input, target, model)

Input: tensor([64.0000,  0.0000, 35.1520,  1.0000,  1.0000,  3.0000])
Target: tensor([50324.4297])
Prediction: tensor([44676.0273])


Are you happy with your model's predictions? Try to improve them further.

## (Optional) Step 6: Try another dataset & blog about it

While this last step is optional for the submission of your assignment, we highly recommend that you do it. Try to clean up & replicate this notebook (or [this one](https://jovian.ml/aakashns/housing-linear-minimal), or [this one](https://jovian.ml/aakashns/mnist-logistic-minimal) ) for a different linear regression or logistic regression problem. This will help solidify your understanding, and give you a chance to differentiate the generic patters in machine learning from problem-specific details.

Here are some sources to find good datasets:

- https://lionbridge.ai/datasets/10-open-datasets-for-linear-regression/
- https://www.kaggle.com/rtatman/datasets-for-regression-analysis
- https://archive.ics.uci.edu/ml/datasets.php?format=&task=reg&att=&area=&numAtt=&numIns=&type=&sort=nameUp&view=table
- https://people.sc.fsu.edu/~jburkardt/datasets/regression/regression.html
- https://archive.ics.uci.edu/ml/datasets/wine+quality
- https://pytorch.org/docs/stable/torchvision/datasets.html

We also recommend that you write a blog about your approach to the problem. Here is a suggested structure for your post (feel free to experiment with it):

- Interesting title & subtitle
- Overview of what the blog covers (which dataset, linear regression or logistic regression, intro to PyTorch)
- Downloading & exploring the data
- Preparing the data for training
- Creating a model using PyTorch
- Training the model to fit the data
- Your thoughts on how to experiment with different hyperparmeters to reduce loss
- Making predictions using the model

As with the previous assignment, you can [embed Juptyer notebook cells & outputs from Jovian](https://medium.com/jovianml/share-and-embed-jupyter-notebooks-online-with-jovian-ml-df709a03064e) into your blog. 

Don't forget to share your work on the forum: https://jovian.ml/forum/t/share-your-work-here-assignment-2/4931

In [0]:
jovian.commit(project=project_name, environment=None)
jovian.commit(project=project_name, environment=None) # try again, kaggle fails sometimes

<IPython.core.display.Javascript object>

[jovian] Attempting to save notebook..
[jovian] Please enter your API key ( from https://jovian.ml/ ):
API KEY: ········
[jovian] Creating a new project "aakashns/02-insurance-linear-regression"
[jovian] Uploading notebook..
[jovian] Committed successfully! https://jovian.ml/aakashns/02-insurance-linear-regression


<IPython.core.display.Javascript object>